# Basics of Mobile Robotics Project -  Main Notebook
### Catarina Pires, Dimitri Hollosi, Goncalo Gomes, Richard Gao 342177
Add Sciper to names


## General Considerations


Here the main code structure and functionalities aim to be detailed so as to gain an understanding in the underlying structure behind the Thymio implementations. Generally speaking, the code can be split into 3 main parts, namely Camera Vision, Global Navigation and Local Navigation. The latter includes all aspects related to obstacle avoidance, which was iteratively tuned to obtain desired performance. One important element to consider prior to delving into the implemented functionalities within the code would be the chosen environment setup for the final project implementation. Indeed, it was decided to use one of of the team member's lower wardrobe portion so as to benefit from a relatively "stable" environment, which could further simplifiy the calibration, lighting and initial setup process. This environment does however come with its limitations, as this implies that the Thymio must navigate in a restricted and confined space, namely implying that the local navigation must take that into account (i.e walls and corners must be accounted for) and yield a relatively high responsiveness.

## Import Required Libraries and Functions

Use this cell to write about what the imported libraries and functions are, especially if they're uncommon.

In [ ]:
import numpy as np
import cv2 as cv
from tdmclient import ClientAsync
from Local_Nav import Local_Nav as LN
from Global_Nav import Global_Nav 
import sys
sys.setrecursionlimit(5000)

## Set Thymio Parameters

Use this cell to write about what these parameters are and why they are chosen.

In [ ]:
r = 2 
L = 9.5 
v_max = 20 
cruising = 150 
motor_speed_max = 500
omega_to_motor = (motor_speed_max*r)/v_max
pixel_to_cm = 1/6

## Set PD Controller Parameters

In order to reach it's goal, the Thymio must convert the difference between it's current position and orientation and that of it's goal into a control action. A PD controller is used here which produces a control action proportional to this difference and damped so that the control action does not oscillate due to overshooting. This method was chosen as it was found through testing that a P or proportional control alone produced undesirable oscillation in the response while a more complex PID control was unnecessary as we experienced little to no steady state error which would require an integral gain to correct. The proportional gain **Kp**, derivative gain **Kd** and **previous_error** (used to calculate the error difference between time steps and hence the damping term along with **Kd**) are initialised in the code cell below. The values of **Kp** and **Kd** were found through manual tuning of the controller until a desired output was shown.

The PDcontroller function can be found as a class method in the Global_Nav.py file. This function takes as input the angle between the vector created by the midpoint of the Thymio to the goal and the vector that runs straight down the centre of the Thymio. The output of the function is the angular velocity of the left and right wheels of the Thymio which are sent to the Thymio at each timestep. The goal of the function is to minimise the angle, thought of as the error in this case, thus turning the Thymio towards the goal. To create the control action, the angular velocity of each wheel is described in terms of a design speed and the error term which has been proportionally gained and damped. The error term is added or subtracted to the design speed of each wheel so as to have an opposite effect on each wheel thereby causing the Thymio to rotate in order to minimise the error and turn the Thymio toward the goal at the design speed. 

In [ ]:
Kp = 0.7                      # Proportional gain                    
Kd = 0.05                     # Derivative gain
previous_error = 0            # Derivative error

## Set Kalman Filter Parameters

In complex systems such as mobile robots, the state of the sysem is almost impossible to determine with complete accuracy. Sensor data of the system can be used to provide a better estimation of the system state and typically the data is passed through a filter when the variables of interest cannot be measured directly. In the case of the Thymio, it's position and orientation can be reliably found using the camera but, when the camera is obscured, we must rely on the Thymio motor speed sensors to infer the new position and orientation at each timestep. Several types of filter exist but the Kalman filter is widely used in robotics for tracking and prediction applications as it is well suited to systems with uncertainty about an observable dynamic process i.e. the motor speeds. 

The Kalman filter is programmed as a recursive function using the estimated state from the previous time step and the sensor data to estimate the current state. At each time step the state transition matrix **A** is used to predict the state variables at the current time step from their previous values. The state covariance **P**, initialised as **P0**, is also predicted using **A**. The observed sensor data at the current time step is then used to calculate the Kalman gain in conjunction with the error in the measurement **R** and the error in the estimate **Q**. Finally the Kalman gain is used to update the prediction of the state and the covariance towards a belief that lies between the initial prediction and the measurement.

The variables used in the Kalman filter function are initialised in the code cell below and the function itself can be found as a class method in the Global_Nav.py file. The covariance function used to create the covariance matrices can also be found in this file. The state transition matrix **A** is constructed so as progress the coordinates and orientation of the Thymio by a single timestep given their previous rates of change. The covariance matrices are constructed so that each element is an error corresponding to the state variable which it involves. These initial values are chosen from the sample values given in the Week 8 Exercise session on Kalman filter implementation.

In [ ]:
ts = 0.1                                                           # time step (s)

A = np.array([[1, 0, 0, 0, 0, 0],     # xdot                       # state transition matrix
              [ts, 1, 0, 0, 0, 0],    # x
              [0, 0, 1, 0, 0, 0],     # ydot
              [0, 0, ts, 1, 0, 0],    # y
              [0, 0, 0, 0, 1, 0],     # phidot
              [0, 0, 0, 0, ts, 1]])   # phi

P0 = GN.covariance(100, 100, 100, 100, 100, 100)                   # state covariance matrix

Q = GN.covariance(0.0615, 0.004, 0.0615, 0.004, 0.0615, 0.004)     # process noise covariance matrix

R = GN.covariance(0.0615, 0.025, 0.0615, 0.025, 0.0615, 0.025)     # measurement covariance matrix

## Begin Thymio Connection and Initialise the Global Navigation Class Object

Use this cell to describe how we communicate with Thymio and what the Global_Nav class is. Make sure to explain why we write the Global_Nav class in this way.

In [ ]:
client = ClientAsync()
node = client.aw(client.wait_for_node())

# Define global_nav object with class attributes
GN = Global_Nav(client, node, cruising, ts, Kp, Kd, previous_error, r, L, omega_to_motor, pixel_to_cm, R, Q, A, P0)

## Vision and Path Creation

Use this cell to describe how we filter/augment the camera image and how the shortest path is created. Make sure to explain why these methods are chosen.

In [3]:
"""Use this cell to write the code for Vision and Path Creation"""

'Use this cell to write the code for Vision and Path Creation'

## Controlling the Thymio and Going to the Goal

Use this cell to describe how the Thymio is controlled in the "main" loop. We might split this into more sections.

In [ ]:
"""Use this cell to write the code for actually sending the Thymio to the goal"""